# Generating methods section of BIDS dataset

Uses pybids and chevron (a mustache template renderer) to automatically generate a method section

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import chevron
import os

import nibabel as nib

from bids import BIDSLayout
from bids.tests import get_test_data_path

from rich import print

import parameters

In [2]:
partials_path = "/home/remi/github/BIDS-methods/partials/"

In [3]:
def highlight_missing_tags(foo):
    foo = "[blue]" + foo + "[/blue]"
    foo = foo.replace("{{", "[/blue][red]{{")
    foo = foo.replace("}}", "}}[/red][blue]")
    return foo

In [4]:
def nb_runs(files):
    all_runs = sorted(list(set([f.get_entities().get("run", 1) for f in files])))
    return len(all_runs)

## Using pybids test data

In [5]:
dataset = "synthetic"

# Here we're using an example BIDS dataset that's bundled with the pybids tests
data_path = os.path.join(get_test_data_path(), dataset)

# Load the BIDS dataset
layout = BIDSLayout(data_path)

### BOLD

In [6]:
# get the first bold file of subject 01
suffix = "bold"
subject = "01"
task = "nback"

files = layout.get(subject=subject, task=task, suffix=suffix, extension=".nii.gz")

this_file = files[0]

metadata = this_file.get_metadata()

filename = this_file.path
img = nib.load(filename)
image_size = parameters.describe_image_size(img)

info = {**metadata, **image_size}

info["suffix"] = this_file.get_entities()["suffix"]
info["repetition_time"] = parameters.describe_repetition_time(metadata)
info["nb_runs"] = nb_runs(files)

duration, nb_vols = parameters.describe_duration(files)
info["duration"] = duration
info["nb_vols"] = nb_vols

Using the `keep` option renders the empty tags

In [7]:
with open("partials/func.mustache", "r") as template:

    args = {
        "template": template,
        "data": info,
        "partials_path": partials_path,
        "keep": True,
    }

    foo = chevron.render(**args)

print(highlight_missing_tags(foo))

The data acquisition was performed in the {{ InstitutionName }}, {{ 
InstitutionalDepartmentName }}, {{ InstitutionAddress }}.

MRI scans were acquired at {{ MagneticFieldStrength }} Tesla using a a {{ 
ManufacturersModelName }} system from {{ Manufacturer }},
with serial number {{ DeviceSerialNumber }}. The software version was {{ SoftwareVersions }}.


For the N-Back task, 2 run(s) of bold
 {{ PulseSequenceType }} {{ ScanningSequence }} {{ SequenceVariant }}
 fMRI data were collected.

The acquisiton parameters were: {{ nb_slices }} slices acquired in a {{ so_str }} fashion; 
repetition time, TR= 2500 ms; flip angle, FA= {{ FlipAngle }}°;

echo time, TE= {{ echo_time }} ms; flip angle, multiband factor= {{ mb_str }};
in-plane acceleration factor= {{ pr_str }}; field of view FOV= 2x2x2 mm; matrix size= 64x64; 
voxel size= 128x128 mm;


Each run was 2:40 minutes in length, during which 64 functional volumes were acquired.

{{ TaskDescription }}

Participants were specifically instructed to: {{ Instructions }}

## Using bids-example

In [8]:
!git ...

git: '...' is not a git command. See 'git --help'.


In [9]:
!ls bids-examples/

7t_trt		   ds007    eeg_ds000117	    pet005
asl001		   ds008    eeg_face13		    qmri_irt1
asl002		   ds009    eeg_hed_small	    qmri_megre
asl003		   ds011    eeg_matchingpennies     qmri_mese
asl004		   ds051    eeg_rest_fmri	    qmri_mp2rage
asl005		   ds052    eeg_rishikesh	    qmri_mp2rageme
bidsconfig.json    ds101    genetics_ukbb	    qmri_mpm
CONTRIBUTING.md    ds102    hcp_example_bids	    qmri_mtsat
ds000001-fmriprep  ds105    ieeg_epilepsy	    qmri_qsm
ds000117	   ds107    ieeg_epilepsy_ecog	    qmri_sa2rage
ds000246	   ds108    ieeg_filtered_speech    qmri_tb1tfl
ds000247	   ds109    ieeg_motorMiller2007    qmri_vfa
ds000248	   ds110    ieeg_visual		    README.md
ds001		   ds113b   ieeg_visual_multimodal  run_tests.sh
ds002		   ds114    pet001		    synthetic
ds003		   ds116    pet002
ds005		   ds210    pet003
ds006		   eeg_cbm  pet004


In [10]:
bids_example_path = os.path.join("/home/remi/github/BIDS-methods/", "bids-examples")

## ASL

In [14]:
dataset = "asl001"

data_path = os.path.join(bids_example_path, dataset)

layout = BIDSLayout(data_path)

suffix = "asl"
subject = "Sub103"

files = layout.get(subject=subject, extension=".nii.gz", suffix=suffix)

this_file = files[0]

metadata = this_file.get_metadata()

info = metadata

info["suffix"] = this_file.get_entities()["suffix"]
info["nb_runs"] = nb_runs(files)

with open("partials/perf.mustache", "r") as template:

    args = {
        "template": template,
        "data": info,
        "partials_path": partials_path,
        "keep": True,
    }

    foo = chevron.render(**args)

print(highlight_missing_tags(foo))

The data acquisition was performed in the {{ InstitutionName }}, {{ 
InstitutionalDepartmentName }}, {{ InstitutionAddress }}.

spiral spiral RM NONE
 perfusion MRI data
were collected ({{ nb_slices }} slices; repetition time, repetition time, TR= {{ 
repetition_time }} ms; flip angle, FA= 111°;
; echo time,
TE= {{ echo_time }} ms; field of view FOV= {{ fov }} mm; matrix size= {{ mat_size }}; voxel 
size= {{ vox_size }} mm;

### EEG

In [12]:
dataset = "eeg_rishikesh"

data_path = os.path.join(bids_example_path, dataset)

layout = BIDSLayout(data_path)

suffix = "eeg"
subject = "001"
task = "meditation"

files = layout.get(subject=subject, task=task, suffix=suffix)

this_file = files[0]

metadata = this_file.get_metadata()

info = metadata

info["suffix"] = this_file.get_entities()["suffix"]
info["nb_runs"] = nb_runs(files)

with open("partials/meeg.mustache", "r") as template:

    args = {
        "template": template,
        "data": info,
        "partials_path": partials_path,
        "keep": True,
    }

    foo = chevron.render(**args)

print(highlight_missing_tags(foo))

The data acquisition was performed in the Paul Sabatier University, Centre de Recherche 
Cerveau et Cognition, Centre de Recherche Cerveau et Cognition, Place du Docteur Baylac, 
Pavillon Baudot, 31059 Toulouse, France.

Recordings were done using a ActiveTwo system from BIOSEMI,
with serial number {{ DeviceSerialNumber }}. The software version was {{ SoftwareVersions }}.


For the meditation task, 1 run(s) of eeg
 data were collected.

Power line and sampling frequencies were 50 Hz and 256 Hz, respectively.

Data was collected with {{ ChannelCount }} eeg channels, 0 EOG channels,
0 ECG channels and 0 EMG channels recorded simultaneously.

{{ TaskDescription }}

Participants were specifically instructed to: {{ Instructions }}